In [1]:
import requests 
import pandas as pd
from pandas.io.json import json_normalize
from random import randint
import numpy as np
import os
import uuid

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from UtilFuncs.get_balance import *
from UtilFuncs.redeem_rewards import * 
from UtilFuncs.process_transactions import * 
from UtilFuncs.return_transactions import *
from UtilFuncs.summarising_history_response import *
from UtilFuncs.create_members import *
from UtilFuncs.cancel_redeem import * 
from UtilFuncs.summarising_outcomes_2 import *
from UtilFuncs.services_logs import * 

# Adhoc testing 
from UtilFuncs.process_transactions_error import * 


Redeem

MapCerts

SubmitPurchase


## Steps

### 1. Create member and assign Rewards - Set the data in Resources/MembersCreate.csv
* A new customer will be created using the input provided in the csv. If we want to use an existing Member, we will need to hardcode the details in the below cell

### 2. Read Resources/TestScenarios file for test scenarios
Config parameters
* TestId: 
* Process: Currently 2 processes are supported - Cancel/Purchase/Return/ExceptionScenarios
* OrderTotal: The total cost of the order. This can be used to control if AT is to be used in the purchase

### 3. For every test the outcome is to be reported as follows: 
* Balances will be printed at the start of the test and after every API call. 
The balances retrieved from GetHistory and GetBalance APIs need to be compared for inconsistencies 
* Print test outcomes. For each test scenario, store the outcomes in a text file: <Outcomes/Customer_name/>
* History data will be exported to excel in a folder Outcomes/<Customer_Name>

---
# Start of Processing
--- 

### 1. Create member and assign rewards

In [2]:
member_details = create_member_and_assign_rewards()

# Return with error if profile_id is empty
if len(member_details) == 0:
    print("Error in creating the member record, using an existing member for development testing")
    profile_id = 'e21f61ff-1990-488b-bfb9-80ef62b270bf'     # Defaulting to a profile id for development
    customer_name = 'Certs-testing-1_Cust-two'
    print(f"Cust name: {customer_name}; Profile ID: {profile_id}")
else:
    profile_id, customer_name = member_details

Creating member: Multiple-redeems Cust-one

{'request': '{"FirstName": "Multiple-redeems", "LastName": "Cust-one", "Status": "A", "Addresses": [{"AddressLine1": "319", "AddressLine2": "COMMONWEALTH AVENUE", "City": "Melbourne", "StateCode": "VIC", "CountryCode": "AU", "PostalCode": "3000", "ChannelCode": "DM", "LocationCode": "H", "IsPreferred": true}], "EMAILS": [{"EMAILADDRESS": "Multiple-redeemsCust-one@email.com", "ChannelCode": "EM", "LocationCode": "B", "IsPreferred": true}], "PHONES": [{"PHONENUMBER": "424052221", "ChannelCode": "SM", "LocationCode": "H", "IsPreferred": true}], "SourceCode": "ENABLER", "EnrollChannelCode": "INSTORE", "JsonExternalData": {"CustomAttributes": {"CUSTOM_MKT_EMAIL_CONSENT": false, "CUSTOM_MKT_SMS_CONSENT": true, "CUSTOM_CRD_EMAIL_CONSENT": false, "CUSTOM_CRD_SMS_CONSENT": true, "IS_SUBSCRIPTION_CHANGED": true, "CUSTOM_LAST_UPDATED_SOURCE": "POS"}}}', 'response': '{\n  "ProfileId": "017dbe5b-2f28-4206-bb2e-122be0a54394",\n  "FirstName": "Multiple-rede

### 2. Read Resources/TestScenarios file for test scenarios and carry out the tests

In [3]:
# Read TestScenarios
df_scenarios = pd.read_csv("Resources/TestScenarios.csv")

# Create a new sub directory in Outcomes directory with Customer name and timestamp
dir_path = create_outcome_directory(customer_name)

# Outcomes in Excel. Start with an empty list. DFs corresponding to workspaces will be added in the loop 
excel_list = []

for index, scenario in df_scenarios.iterrows():

    # Create a new text file for writing test scenario outcomes
    outcomes_file_name = dir_path + "/" + str(scenario['TestId']) + ".txt"
    f = open(outcomes_file_name , "w")

    # Creating the services log file 
    create_new_file(f"{dir_path}/services_logs_{str(scenario['TestId'])}.json")

    f.write(f"\nExecuting a {scenario['Scenario']} test with order total as ${scenario['OrderTotal']}")
        
    f.write(f"\n---------------------------Retrieving History for {customer_name} before running the test--------------------")
    try: 
        df_history_before = pd.DataFrame(json_normalize(get_history(profile_id)))

        # Verify if the balance matches with the balance api
        # Print the total rewards
        rewards_balance = get_active_rewards_balance(df_history_before, f)
        if rewards_balance == get_balance(profile_id):
            f.write("\n***CHECK: Rewards balance compared successfully between GetHistory and Balance APIs \n")

    except NotImplementedError as error:
        print("No History data for this customer")


    total_dollar_value = rewards_balance['total_dollar_value']

    temp_order_id = str(randint(0,99999))
    # temp_order_id = str(uuid.uuid4())    

    # Total Order Value in $ - fetched from the TestScenarios.csv file
    order_total_value = scenario['OrderTotal']

    ######## Redeem API

    # Need to add logic later for scenarios where the customer doesnt wish to redeem rewards. For now we can test it with a member who doesnt have any rewards

    # If the total order value > total rewards value, 
    # Redeem rewards worth total rewards value
    # else, redeem rewards worth total order value
    if order_total_value > total_dollar_value:
        rewards_value_for_burn = total_dollar_value
    else:
        rewards_value_for_burn = order_total_value

    f.write(f"\n\n---------------------------Redeeming rewards worth ${str(rewards_value_for_burn)} for the temporary transaction ID {temp_order_id}")
    # redeem_response = redeem_rewards(str(scenario['TestId']), profile_id, rewards_value_for_burn, temp_order_id, f)

    redeem_response = redeem_rewards_dec(
        test_id = str(scenario['TestId']), 
        profile_id = profile_id,
        dollarAmount = rewards_value_for_burn,
        transaction_id = temp_order_id,
        file = f
    )

    f.write(f"\n\n---------------------------Retrieving History for {customer_name} after Redeem API--------------------")
    try: 
        df_history_after_redeem = pd.DataFrame(json_normalize(get_history(profile_id)))

        # Verify if the balance matches with the balance api
        # Print the total rewards
        rewards_balance = get_active_rewards_balance(df_history_after_redeem, f)
        if rewards_balance == get_balance(profile_id):
            f.write("\n***CHECK: Rewards balance compared successfully between GetHistory and Balance APIs \n")

    except NotImplementedError as error:
        print("No History data for this customer")

    #-------------------------------------------------------------------------------------------------------------------------
    ######## Cancellation API
    if scenario['Scenario'] == 'Cancel':
        
        # Calling Cancel API
        f.write(f"\n\n---------------------------Cancelling the above temporary transaction ID {temp_order_id}")
        cancel_redeem(profile_id, temp_order_id, f)

        f.write(f"\n\n---------------------------Retrieving History for {customer_name} after Cancel API--------------------")
        try: 
            df_history_after_cancel = pd.DataFrame(json_normalize(get_history(profile_id)))

            # Verify if the balance matches with the balance api
            rewards_balance = get_active_rewards_balance(df_history_after_cancel, f)
            if rewards_balance == get_balance(profile_id):
                f.write("\n\n\n***CHECK: Rewards balance compared successfully between GetHistory and Balance APIs \n")

        except NotImplementedError as error:
            print("No History data for this customer")

    #-------------------------------------------------------------------------------------------------------------------------
    ######## Submit API
    if scenario['Scenario'] in ['Purchase', 'Return']:

        # [submit_request, submit_response] = purchase_transactions(str(scenario['TestId']), profile_id, temp_order_id,  order_total_value, f)
        purchase_transactions_dec(
            test_id = str(scenario['TestId']),
            profile_id = profile_id,
            temp_ref_no = temp_order_id,
            order_total = order_total_value, 
            file =  f
        )

        f.write(f"\n\n---------------------------Retrieving History for {customer_name} after Submit API for Purchase--------------------")
        try: 
            df_history_after_submit = pd.DataFrame(json_normalize(get_history(profile_id)))

            # Verify if the balance matches with the balance api
            rewards_balance = get_active_rewards_balance(df_history_after_submit, f)
            if rewards_balance == get_balance(profile_id):
                f.write("\n\n\n***CHECK: Rewards balance compared successfully between GetHistory and Balance APIs \n")

        except NotImplementedError as error:
            print("No History data for this customer")

    #-------------------------------------------------------------------------------------------------------------------------
    ######## Submit API - Exception testing - testing the case where the return was submitted first and the purchase was posted afterwards
    if scenario['Scenario'] == 'Return_before_purchase':

        # This function will not call the submit API 
        purchase_transactions_error(str(scenario['TestId']), profile_id, temp_order_id, redeem_response, order_total_value, f)

        # Call the return API and return the first 2 items
        [return_request, return_response] = return_transactions_partial_error(str(scenario['TestId']), profile_id, [0,1], f)

        f.write(f"\n\n---------------------------Retrieving History for {customer_name} after Submit API for Return--------------------")
        try: 
            df_history_after_return = pd.DataFrame(json_normalize(get_history(profile_id)))

        except NotImplementedError as error:
            print("No History data for this customer")

        # input("Hit Enter to proceed with the Purchase transaction")
        
        # Now submit the actual purchase transaction 
        [submit_request, submit_response] = submit_transaction_after_return(str(scenario['TestId']), profile_id, temp_order_id, redeem_response, order_total_value, f)
        f.write(f"\n\n---------------------------Retrieving History for {customer_name} after Submit API for Purchase--------------------")

        try: 
            df_history_after_submit = pd.DataFrame(json_normalize(get_history(profile_id)))

            # Verify if the balance matches with the balance api
            rewards_balance = get_active_rewards_balance(df_history_after_submit, f)
            if rewards_balance == get_balance(profile_id):
                f.write("\n\n\n***CHECK: Rewards balance compared successfully between GetHistory and Balance APIs \n")

        except NotImplementedError as error:
            print("No History data for this customer")

    #-------------------------------------------------------------------------------------------------------------------------
    ######## Return API 
    if scenario['Scenario'] == 'Return':

        if scenario['TestId'] == 1: 
            # Do a partial return of first 2 items
            return_items = [0, 1]
        if scenario['TestId'] == 2:
            # Do a partial return of the 3rd item
            return_items = [2]
        else: 
            # Do a full return
            return_items = [0,1,2]

        [return_request, return_response] = return_transactions_partial(str(scenario['TestId']), profile_id, return_items, f)

        f.write(f"\n\n---------------------------Retrieving History for {customer_name} after Submit API for Return--------------------")
        try: 
            df_history_after_return = pd.DataFrame(json_normalize(get_history(profile_id)))

            # Verify if the balance matches with the balance api
            rewards_balance = get_active_rewards_balance(df_history_after_return, f)
            if rewards_balance == get_balance(profile_id):
                f.write("\n\n\n***CHECK: Rewards balance compared successfully between GetHistory and Balance APIs \n")

        except NotImplementedError as error:
            print("No History data for this customer")

    #-------------------------------------------------------------------------------------------------------------------------
    ######## Summarising Outcomes
        
    # 1. Pass the dictionary containing the history data: 
    #     * before running the test 
    #     * after redeem
    #     * after cancellation (if applicable)
    #     * after submit (if applicable) 
    # 2. Create a Excel workbook with the following sheets:
    #     * Sheet1: Test scenarios
    #     * Sheet2: History data comparison for first test, titled: Test(id)_Rewards_balances
    #     * Sheet3: History data by Expiration dates, titled: Test(id)_Expiration_dates
    #     * Sheets4 onwards: Sheets 2 and sheets 3 for each test

    if scenario['Scenario'] in ['Purchase', 'Return', 'Return_before_purchase']:

        dict_dfs = [
            {
            "Operation": "Before",
            "History": df_history_before
            }, 
            {
            "Operation": "Redeem",
            "History": df_history_after_redeem
            }, 
            {
            "Operation": "Purchase",
            "History": df_history_after_submit
            } 
        ]

        if scenario['Scenario'] in ['Return', 'Return_before_purchase']:
            dict_dfs.append({
                "Operation": "Return",
                "History": df_history_after_return
            })

    elif scenario['Scenario'] == 'Cancel':

        dict_dfs = [
            {
            "Operation": "Before",
            "History": df_history_before
            }, 
            {
            "Operation": "Redeem",
            "History": df_history_after_redeem
            }, 
            {
            "Operation": "Cancel",
            "History": df_history_after_cancel
            }
        ]

    [df_summary, df_expiration] = summarise_outcomes(dict_dfs, scenario['TestId'])

    sheet_name_rewards = "Test" + str(scenario['TestId']) + "_Rewards_balances"
    sheet_name_expiry = "Test" + str(scenario['TestId']) + "_Expiration_dates"
    excel_list += [
        {
            "Name": sheet_name_rewards, 
            "DF": df_summary
        },
        {
            "Name": sheet_name_expiry, 
            "DF": df_expiration
        }
    ]

    # Adding the entire response from GetHistory 
    sheet_name_balance = "Test" + str(scenario['TestId']) + "_Balance"
    if scenario['Scenario'] == 'Cancel':
        excel_list.append({
            "Name": sheet_name_balance, 
            "DF": df_history_after_cancel
        })
    elif scenario['Scenario'] == 'Purchase':
        excel_list.append({
            "Name": sheet_name_balance, 
            "DF": df_history_after_submit
        })
    elif scenario['Scenario'] in ['Return', 'Return_before_purchase']:
        excel_list.append({
            "Name": sheet_name_balance, 
            "DF": df_history_after_submit
        })
        excel_list.append({
            "Name": sheet_name_balance, 
            "DF": df_history_after_return
        })

    f.close()


New directory created for test outcomes: Outcomes/Multiple-redeems_Cust-one_05-24-2023-22-21-31


### 3. Storing outcomes to an Excel Workbook

In [4]:
with pd.ExcelWriter(f'{dir_path}/output.xlsx') as writer:

    df_scenarios.to_excel(writer, sheet_name="Test Scenarios")
    
    for sheet in excel_list:
        sheet['DF'].to_excel(writer, sheet_name=sheet['Name'])


---
## Break processing here, in case we do a run all

In [5]:
# To break processing if we run all 
assert False

AssertionError: 

#### Work area for returns API

In [ ]:
service_logs = read_api_payloads()
# len(service_logs['service_calls'][0])

# Search Service logs for key Api_name = "SubmitPurchase"
calls = [item for item in service_logs['service_calls'] if item.get('Api_name') == 'SubmitPurchase' ]
if len(calls) > 0: submit_call = calls[0]

original_txn_num = submit_call['payload']['request']['TransactionNumber']
original_txn_date = submit_call['payload']['request']['TransactionDateTime']
original_store_code = submit_call['payload']['request']['StoreCode']

print(submit_call['payload']['request'])


In [ ]:
# Refer to the sample json file 
dict_json = json.load(open("Resources/return_transactions.json"))

# Setting header data 

dict_json['ProfileId'] = profile_id
dict_json['TransactionNumber'] = int(datetime.now().timestamp())
dict_json['TransactionDateTime'] = datetime.now()
dict_json['TransactionNetTotal'] = submit_call['payload']['request']['TransactionNetTotal'] * -1
dict_json['TransactionTypeCode'] = 'RT'

# Adding the line items
return_transaction_details = []
for item in submit_call['payload']['request']['TransactionDetails']:
    return_item = item
    return_item['ItemTransactionTypeCode'] = 'RT'
    return_item['DollarValueGross'] *= -1 
    return_item['DollarValueNet'] *= -1 
    return_item['originalTransactionDateTime'] = original_txn_date
    return_item['originalStoreCode']  = original_store_code
    return_item['originalTransactionNumber'] = original_txn_num
    return_transaction_details.append(return_item)

dict_json['TransactionDetails'] = return_transaction_details

dict_json['Tenders'] = []
for tender in submit_call['payload']['request']['Tenders']:
    tender['TenderAmount'] *= -1
    dict_json['Tenders'].append(tender)


payload = json.dumps(dict_json, default= str)

url = "https://u1srgl-pveapi.epsilonagilityloyalty.com/api/v2/transaction"

headers = {
'Accept-Language': 'en-US',
'Authorization': 'OAuth ' + get_token(),
'Content-Type': 'application/json',
'Program-Code': 'REBEL'
}

print(payload)
response = requests.request("POST", url, headers={
                                                'Accept-Language': 'en-US',
                                                'Authorization': 'OAuth ' + get_token(),
                                                'Content-Type': 'application/json',
                                                'Program-Code': 'REBEL' 
                                                }, 
                            data=payload)

if response.status_code == 200:
    print("REturn transaction was successfully posted")

else: 
    print("Return transaction was unsuccessful")
    print(response.text)
